# String Matching
The treatment of string attributes in a database requires further discussion. For string attributes,
databases are good at looking up or joining on exact matches using hashing. For more complicated
conditions (e.g., find all near matches in two tables), this story is a little more complicated. We start
our study of string-searching algorithms, sometimes called string-matching algorithms. These are
an important class of string algorithms that try to find a place where one or several strings (also
called patterns) are found within a larger set of strings.

## Similarity Metric
A similarity metric is a function that assigns a score to two strings to evaluate how similar they are. For example, the string "University of Chicago" and "Chicago" should be more similar than "Apple" and "Bannana". String comparison is a core primitive in data science because having consistent representations is important for programming. 

One way to design a similarity metric is to consider the tokens in a string. The first step in designing a good string similarity metric is the process of tokenization. This considers the minimum granularity of matching that we will consider. Formally, tokenization takes the string which is a sequence of characters and turns it into a set of tokens. Think of these like the important features of a string that are worth considering. For example, we might tokenize a string on word boundaries:

In [44]:
st = 'the quick brown fox'
set(st.split())

{'brown', 'fox', 'quick', 'the'}

Or we might tokenize on pairs of sucessive words:

In [45]:
st = 'the quick brown fox'
words = st.split()
set(zip(words[:-1], words[1:]))

{('brown', 'fox'), ('quick', 'brown'), ('the', 'quick')}

Our similarity metric will compare how many tokens are shared between two strings. 

## Jaccard Similarity
Over a sequence of tokens one of the simplest (but ubiquitous!) similarity measures is Jaccard
Similarity. Jaccard similarity, also known as Intersection over Union, is defined as the size of the
intersection (how many unique tokens appear in both) divided by the size of the union (how many
unique tokens appear in either). This results in a metric from 0 to 1 where 0 indicates no overlapping tokens and 1 indicates all of the tokens overlap.

In [52]:
def tokenize(st):
    return set(st.lower().split())

def jaccard(a,b):
    Ta = tokenize(a)
    Tb = tokenize(b)
    
    return len(Ta.intersection(Tb))/len(Ta.union(Tb))

In [53]:
jaccard('Apple', 'Apple Pie')

0.5

In [54]:
jaccard('Apple Pie', 'Pie Apple Pie')

1.0

In [55]:
jaccard('Banana', 'Apple Pie')

0.0

We can build a matcher that considers Jaccard Similarities and identifies similar strings in two lists, namely, find all pairs that are within a specific similarity of each other:

In [58]:
strlist1 = [ 'the big bear' , 'dog in the woods' , 'alphabet soup' , 'kermit surprise' ]
strlist2 = ['big bear', 'dogs', 'alphabet soup 1', 'kermit surprise' ]


def match(l1, l2, threshold):
    matches = []
    
    for i in l1:
        for j in l2:
            if jaccard(i,j) >= threshold:
                matches.append((i,j))
    return matches

print('J>=1', match(strlist1, strlist2, 1.0))
print('J>=0.5', match(strlist1, strlist2, 0.5))
print('J>=0.0', match(strlist1, strlist2, 0.0))

J>=1 [('kermit surprise', 'kermit surprise')]
J>=0.5 [('the big bear', 'big bear'), ('alphabet soup', 'alphabet soup 1'), ('kermit surprise', 'kermit surprise')]
J>=0.0 [('the big bear', 'big bear'), ('the big bear', 'dogs'), ('the big bear', 'alphabet soup 1'), ('the big bear', 'kermit surprise'), ('dog in the woods', 'big bear'), ('dog in the woods', 'dogs'), ('dog in the woods', 'alphabet soup 1'), ('dog in the woods', 'kermit surprise'), ('alphabet soup', 'big bear'), ('alphabet soup', 'dogs'), ('alphabet soup', 'alphabet soup 1'), ('alphabet soup', 'kermit surprise'), ('kermit surprise', 'big bear'), ('kermit surprise', 'dogs'), ('kermit surprise', 'alphabet soup 1'), ('kermit surprise', 'kermit surprise')]


## Min Hash Algorithm

We will now consider an approximate Jaccard matcher which is significantly faster. The MinHash algorithm relies on the fact that the Jaccard Similarity is sort of like a probability---if I randomly draw an element from the union, what is the probability that it will be in the intersection? We can use this insight to develop a fast approximate algorithm. The first thing that we will need is a generator that can generate k independent hash functions.

In [59]:
import binascii
import random

def doHash(st, a,b):
    NEXTPRIME = 4294967311
    val = binascii . crc32( bytes ( str (st), 'utf-8' ))
    return (a * val + b) % NEXTPRIME

def generateHash(k):
    MAXINT = 2**32 - 1
    return [ (random.randint(0,MAXINT), random.randint(0,MAXINT)) for i in range(k)]

For each hash function, we compute the minimum value of the hash function over all possible tokens.

In [60]:
def minhash(st, a,b):
    tokens = tokenize(st)
    return min([doHash(t, a, b) for t in tokens])

def signature(st, hashes):
    return [minhash(st,a,b) for a,b in hashes]

hashes = generateHash(10)

print(signature('kermit surprise', hashes))
print(signature('kermit surprise 1', hashes))
print(signature('bear brown', hashes))

[1771224429, 2573226673, 3443466068, 2394302545, 1944151189, 330307586, 380011922, 1691827482, 150724341, 1166715779]
[1052982803, 2435591847, 3426789228, 735019560, 1944151189, 330307586, 380011922, 404624650, 150724341, 1166715779]
[2037520169, 1042073495, 2726826241, 321601117, 979332860, 3750545969, 208957705, 1042412753, 3372274495, 950563002]


If we look at the results carefully the strings that have a greater overlap have a higher probability of minhashes matching for randomly chosen hash functions!

## Edit Distance

Jaccard similarity is a pretty naive metric–it does not consider the order of words and it does not
do well with spelling errors. However, it admits an O(n) time approximation algorithm. In this
lecture, we will consider the opposite extreme. A much more comprehesive string similarity met-
ric but one that is much harder to compute. 

### Levenshtein (Edit) Distance
The Levenshtein distance is a string metric for measuring the difference between two sequences.
Informally, the Levenshtein distance between two words is the minimum number of single-
character edits (insertions, deletions or substitutions) required to change one word into the other.
Let see a few examples that have an edit distance of 1.

In [24]:
strs = ('abc', 'ab') #delete c
strs = ('abc','abd') #substitute c for d
strs = ('abc', 'abdc') #insert d

A more complicated example is:

In [25]:
strs = ('bac', 'abd') #the edit distance is 3 (insert b, delete b, subtitute c)

Let's write a simple recursive algorithm to compute edit distance:

In [28]:
def edit(s,t):
    
    if s == "":
        return len(t)
    
    if t == "":
        return len(s)
    
    return min([1 + edit(s[1:], t), \
                1 + edit(s, t[1:]), \
                (s[ 0 ] != t[ 0 ]) + edit(s[ 1 :], t[ 1 :])])
    

In [61]:
edit('kitten', 'sitting')

3

In [63]:
edit('kittens the', 'sitting on')

6

In [64]:
def editp(s,t):
    
    print('Comparing', s,t)
    
    if s == "":
        return len(t)
    
    if t == "":
        return len(s)
    
    return min([1 + editp(s[1:], t), 1 + editp(s, t[1:]), (s[ 0 ] != t[ 0 ]) + editp(s[ 1 :], t[ 1 :])])

editp('bac', 'abd')

Comparing bac abd
Comparing ac abd
Comparing c abd
Comparing  abd
Comparing c bd
Comparing  bd
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing  d
Comparing  bd
Comparing ac bd
Comparing c bd
Comparing  bd
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing  d
Comparing ac d
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing ac 
Comparing c 
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing c bd
Comparing  bd
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing  d
Comparing bac bd
Comparing ac bd
Comparing c bd
Comparing  bd
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing  d
Comparing ac d
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing ac 
Comparing c 
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing bac d
Comparing ac d
Comparing c d
Comparing  d
Comparing c 
Comparing  
Comparing ac 
Comparing c 
Comparing bac 
Comparing ac 
Comparing ac d
Comparing c d
Comparing  d
Comparing c 
Comparing

3

In [65]:
memoization = {}

def editfast(s,t):
    
    if (s,t) in memoization:
        return memoization[(s,t)]
    
    if s == "":
        return len(t)
    
    if t == "":
        return len(s)
    
    rtn = min([1 + editfast(s[1:], t), 1 + editfast(s, t[1:]), (s[ 0 ] != t[ 0 ]) + editfast(s[ 1 :], t[ 1 :])])
    
    memoization[(s,t)] = rtn
    
    return rtn

editfast('kittens the', 'sitting on test')

9